<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/How_to_Train_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data**

In [1]:
!wget http://www.gutenberg.org/cache/epub/16457/pg16457.txt

--2022-09-29 03:40:32--  http://www.gutenberg.org/cache/epub/16457/pg16457.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/16457/pg16457.txt [following]
--2022-09-29 03:40:32--  https://www.gutenberg.org/cache/epub/16457/pg16457.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617622 (603K) [text/plain]
Saving to: ‘pg16457.txt’

pg16457.txt         100%[===================>] 603.15K  --.-KB/s    in 0.1s    

2022-09-29 03:40:32 (4.99 MB/s) - ‘pg16457.txt’ saved [617622/617622]



In [2]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip

--2022-09-29 03:40:47--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.42.152
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.42.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M   100MB/s    in 1.8s    

2022-09-29 03:40:49 (100 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]



In [3]:
!unzip wikitext-103-raw-v1.zip

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 45.5 MB/s 
     |████████████████████████████████| 6.6 MB 34.9 MB/s 


In [5]:
import transformers

In [6]:
## importing the tokenizer and subword BPE trainer
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer

## a pretokenizer to segment the text into words
from tokenizers.pre_tokenizers import Whitespace